<a href="https://colab.research.google.com/github/Tony20105972/Samantha_OS_Backend/blob/main/LangGraph_Flow_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes -q
!pip install --upgrade huggingface_hub -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.3 MB/s eta 0:00:00


In [ ]:
# phi3_langgraph_runner.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import TypedDict, Optional
import os

# 1. ✅ 상태 정의
class AgentState(TypedDict):
    prompt: str
    result: Optional[str]

# 2. ✅ 모델 로딩 (SaaS 수준 성능 최적화 포함)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-3-mini-4k-instruct",
    device_map="auto"
)
model.config.attn_implementation = "eager"  # 호환성 확보

# 3. ✅ Node 정의 (LangGraph 스타일)
def phi3_node(state: AgentState) -> AgentState:
    prompt = state["prompt"]
    if not prompt:
        return {**state, "result": "❌ No prompt provided."}

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    try:
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
        )
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return {**state, "result": result}
    except Exception as e:
        return {**state, "result": f"❌ Error: {str(e)}"}

# 4. ✅ 헌법 검사기 (선택)
def rule_checker(state: AgentState) -> AgentState:
    # 예: 금지어 검사 등
    forbidden = ["violence", "harm"]
    if any(word in (state["result"] or "").lower() for word in forbidden):
        return {**state, "result": "❌ Rule Violation Detected."}
    return state

# 5. ✅ 외부 출력 처리기
def external_output(state: AgentState):
    print("✅ Final Output:", state["result"])
    # 예: Slack 전송, HTML 저장 등 SaaS용 확장 가능

# 6. ✅ 실행 흐름 (LangGraph 스타일)
def run_agent_flow(prompt: str):
    state: AgentState = {"prompt": prompt, "result": None}
    state = phi3_node(state)
    state = rule_checker(state)
    external_output(state)
    return state



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import json
import uuid
import time
from collections import deque

class LangGraphRunner:
    """
    LangGraph 흐름을 시뮬레이션하고 실행하는 클래스입니다.
    사용자가 정의한 노드와 엣지를 기반으로 에이전트 워크플로우를 실행합니다.
    """

    def __init__(self, flow_filepath='sample_flow.json', constitution_filepath='constitution.json'):
        """
        초기화 메서드: 흐름 정의와 헌법(규칙)을 로드합니다.

        Args:
            flow_filepath (str): LangGraph 흐름 정의가 담긴 JSON 파일 경로.
            constitution_filepath (str): 헌법(규칙) 정의가 담긴 JSON 파일 경로.
        """
        self.flow_definition = self._load_json(flow_filepath)
        self.constitution = self._load_json(constitution_filepath)
        self.nodes = {node['id']: node for node in self.flow_definition.get('nodes', [])}
        self.edges = self.flow_definition.get('edges', [])
        print(f"흐름 정의 로드 완료: {len(self.nodes)}개 노드, {len(self.edges)}개 엣지")
        print(f"헌법 정의 로드 완료: {len(self.constitution.get('rules', []))}개 규칙")

    def _load_json(self, filepath):
        """JSON 파일을 로드합니다."""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"오류: 파일을 찾을 수 없습니다 - {filepath}")
            return {}
        except json.JSONDecodeError:
            print(f"오류: JSON 디코딩 실패 - {filepath}")
            return {}

    def _execute_node(self, node_id, current_state, execution_log):
        """
        단일 노드의 로직을 실행하고 상태를 업데이트합니다.
        (이 부분은 시뮬레이션을 위한 더미 로직입니다.)

        Args:
            node_id (str): 현재 실행할 노드의 ID.
            current_state (dict): 현재 워크플로우의 상태 변수 딕셔너리.
            execution_log (list): 실행 로그를 기록할 리스트.

        Returns:
            dict: 업데이트된 상태 변수.
            str: 다음 노드로의 라우팅 결정 (ROUTER 노드 전용).
            bool: 규칙 위반 여부 (RULE_CHECKER 노드 전용).
        """
        node = self.nodes.get(node_id)
        if not node:
            print(f"경고: 노드를 찾을 수 없습니다 - {node_id}")
            return current_state, None, False

        node_type = node['type']
        node_label = node['label']
        properties = node.get('properties', {})
        output_vars = properties.get('outputVars', '')
        input_vars = properties.get('inputVars', '')

        print(f"\n--- 노드 실행: [{node_type}] {node_label} (ID: {node_id}) ---")
        log_entry = {
            "timestamp": time.time(),
            "node_id": node_id,
            "node_type": node_type,
            "node_label": node_label,
            "input_state_before_execution": current_state.copy(),
            "output_data": {}
        }

        next_route = None
        rule_violation = False

        if node_type == 'STATE':
            # 초기 상태 변수 설정 (예: initial_topic)
            for key, value in properties.items():
                if key.startswith('initial_'):
                    var_name = key[len('initial_'):]
                    current_state[var_name] = value
                    log_entry["output_data"][var_name] = value
            print(f"초기 상태 설정: {current_state}")

        elif node_type == 'LLM':
            prompt_template = properties.get('promptTemplate', '')
            # 입력 변수 가져오기
            input_values = {var: current_state.get(var, f"<{var} 없음>") for var in input_vars.split(',') if var}
            # 프롬프트 템플릿 채우기 (간단한 시뮬레이션)
            generated_content = prompt_template
            for var, val in input_values.items():
                generated_content = generated_content.replace(f"{{{var}}}", str(val))

            # LLM 결과 시뮬레이션
            llm_output = f"생성된 콘텐츠: '{generated_content}' (온도: {properties.get('temperature')})"
            if output_vars:
                current_state[output_vars] = llm_output
                log_entry["output_data"][output_vars] = llm_output
            print(f"LLM 출력: {llm_output}")

        elif node_type == 'TOOL':
            tool_name = properties.get('toolName', '기본 도구')
            params_str = properties.get('params', '{}')
            try:
                params = json.loads(params_str)
            except json.JSONDecodeError:
                params = {}
            input_values = {var: current_state.get(var, f"<{var} 없음>") for var in input_vars.split(',') if var}

            # 도구 실행 시뮬레이션
            tool_result = f"'{tool_name}' 도구 실행 완료. 입력: {input_values}, 파라미터: {params}"
            if output_vars:
                current_state[output_vars] = tool_result
                log_entry["output_data"][output_vars] = tool_result
            print(f"도구 출력: {tool_result}")

        elif node_type == 'ROUTER':
            condition_logic = properties.get('conditionLogic', '')
            input_value_for_router = current_state.get(input_vars, None) # 라우터 입력 변수

            # 라우터 조건 로직 시뮬레이션
            # 실제로는 복잡한 조건 파싱 및 평가 필요
            if "if" in condition_logic and "==" in condition_logic and "->" in condition_logic:
                try:
                    # 예시: "if is_valid == false -> to node_llm_rewrite"
                    condition_part = condition_logic.split("->")[0].strip()
                    decision_part = condition_logic.split("->")[1].strip()

                    # 간단한 조건 평가 시뮬레이션 (is_valid == false)
                    if "is_valid == false" in condition_part and input_value_for_router == False:
                        next_route = "invalid" # 예시 라우팅 결정
                    elif "is_valid == true" in condition_part and input_value_for_router == True:
                        next_route = "valid" # 예시 라우팅 결정
                    else:
                        next_route = "default" # 조건 불일치 시 기본 경로
                except Exception as e:
                    print(f"라우터 로직 파싱 오류: {e}")
                    next_route = "error"
            else:
                next_route = "default" # 기본 라우팅

            if output_vars:
                current_state[output_vars] = next_route
                log_entry["output_data"][output_vars] = next_route
            print(f"라우터 결정: {next_route}")

        elif node_type == 'RULE_CHECKER':
            rule_set_name = properties.get('ruleSetName', '')
            input_value_for_rule = current_state.get(input_vars, '')

            # 헌법(규칙)을 기반으로 규칙 검사 시뮬레이션
            # 실제로는 constitution.json의 'logic' 필드를 파싱하여 평가해야 함
            rule_found = False
            for rule in self.constitution.get('rules', []):
                if rule['name'] == rule_set_name:
                    rule_found = True
                    # 시뮬레이션을 위해 'no_bias' 규칙은 항상 위반으로 가정
                    # 'content_length' 규칙은 입력 길이에 따라 다르게 가정
                    if rule_set_name == "no_bias":
                        rule_violation = True # 항상 위반으로 시뮬레이션
                        print(f"규칙 '{rule_set_name}' 검사: 위반 (시뮬레이션)")
                    elif rule_set_name == "content_length":
                        if isinstance(input_value_for_rule, str) and len(input_value_for_rule.split()) < 500:
                            rule_violation = True
                            print(f"규칙 '{rule_set_name}' 검사: 위반 (콘텐츠 길이 부족)")
                        else:
                            rule_violation = False
                            print(f"규칙 '{rule_set_name}' 검사: 통과 (콘텐츠 길이 충분)")
                    else:
                        rule_violation = False # 다른 규칙은 통과로 시뮬레이션
                        print(f"규칙 '{rule_set_name}' 검사: 통과 (기본)")
                    break
            if not rule_found:
                print(f"경고: 규칙 세트 '{rule_set_name}'를 헌법에서 찾을 수 없습니다. 기본적으로 통과 처리.")
                rule_violation = False

            is_valid = not rule_violation # is_valid는 위반이 아니면 True
            if output_vars:
                current_state[output_vars] = is_valid
                log_entry["output_data"][output_vars] = is_valid
            print(f"규칙 검사 결과 (is_valid): {is_valid}")


        elif node_type == 'OUTPUT':
            output_type = properties.get('outputType', 'Webhook')
            target_url = properties.get('targetUrl', '')
            send_vars = properties.get('sendVars', '')
            output_data = {var: current_state.get(var, f"<{var} 없음>") for var in send_vars.split(',') if var}

            # 최종 출력 시뮬레이션
            print(f"최종 출력 노드: 타입={output_type}, 대상={target_url}, 데이터={output_data}")
            log_entry["output_data"] = output_data

        log_entry["output_state_after_execution"] = current_state.copy()
        execution_log.append(log_entry)
        return current_state, next_route, rule_violation

    def run_flow(self, initial_inputs={}):
        """
        정의된 흐름을 시작부터 끝까지 실행합니다.

        Args:
            initial_inputs (dict): 흐름 시작 시 주입할 초기 입력 변수.

        Returns:
            dict: 최종 상태 변수.
            list: 전체 실행 로그.
            list: 규칙 위반이 발생한 엣지 목록.
        """
        current_state = initial_inputs.copy()
        execution_log = []
        violated_edges = []

        # 시작 노드 찾기 (STATE 타입 노드를 시작 노드로 간주)
        start_node_id = None
        for node_id, node_data in self.nodes.items():
            if node_data.get('type') == 'STATE':
                start_node_id = node_id
                break

        if not start_node_id:
            print("오류: 시작 STATE 노드를 찾을 수 없습니다. 흐름을 시작할 수 없습니다.")
            return current_state, execution_log, violated_edges

        current_node_id = start_node_id
        visited_nodes = set() # 무한 루프 방지

        # BFS (너비 우선 탐색) 또는 DFS (깊이 우선 탐색) 대신,
        # 단순 선형 흐름을 가정하고 라우터 노드에서 분기 처리
        # 복잡한 그래프 순회는 실제 LangGraph 로직이 필요

        # 현재는 시작 노드부터 엣지를 따라 순차적으로 실행하는 방식으로 시뮬레이션
        # 실제 LangGraph는 더 복잡한 그래프 순회 로직을 가짐.
        # 이 시뮬레이션은 'Router' 노드를 통한 조건부 분기만 처리

        # 현재 노드에서 다음 노드를 찾기 위한 큐
        queue = deque([current_node_id])

        while queue:
            node_to_execute_id = queue.popleft()

            if node_to_execute_id in visited_nodes:
                # 이미 방문한 노드 (루프 발생 가능성)
                # 이 시뮬레이션에서는 단순 루프를 방지하지만, 실제 LangGraph는 더 복잡한 루프 처리 로직 가짐
                print(f"경고: 노드 '{node_to_execute_id}'는 이미 방문했습니다. 루프 방지를 위해 건너뜁니다.")
                continue

            visited_nodes.add(node_to_execute_id)

            node_data = self.nodes.get(node_to_execute_id)
            if not node_data:
                print(f"오류: 존재하지 않는 노드 ID: {node_to_execute_id}")
                continue

            current_state, router_decision, rule_violation_flag = self._execute_node(
                node_to_execute_id, current_state, execution_log
            )

            if rule_violation_flag:
                # 규칙 위반이 발생한 엣지를 찾아서 기록 (시뮬레이션)
                # 실제로는 해당 노드와 연결된 모든 엣지를 위반으로 표시할 수 있음
                for edge in self.edges:
                    if edge['source'] == node_to_execute_id:
                        violated_edges.append(edge['id'])
                        print(f"규칙 위반 엣지 기록: {edge['id']}")
                        break # 첫 번째 위반 엣지만 기록 (예시)

            # 다음 노드 결정 로직
            next_edges = [e for e in self.edges if e['source'] == node_to_execute_id]

            if node_data['type'] == 'ROUTER':
                # 라우터 노드는 결정에 따라 다음 엣지를 선택
                found_next_edge = False
                for edge in next_edges:
                    condition = edge.get('condition')
                    if condition:
                        # 조건 파싱 및 평가 (간단한 시뮬레이션)
                        # 예: "decision_path == 'valid'"
                        if f"decision_path == '{router_decision}'" in condition:
                            queue.append(edge['target'])
                            print(f"라우터 결정에 따라 다음 노드: {edge['target']}")
                            found_next_edge = True
                            break
                    # 조건이 없거나 조건에 맞지 않으면 기본 엣지 (첫 번째 엣지)
                if not found_next_edge and next_edges:
                    # 라우터 조건에 맞는 엣지가 없으면 첫 번째 엣지를 기본으로 선택
                    queue.append(next_edges[0]['target'])
                    print(f"라우터 조건 불일치, 기본 다음 노드: {next_edges[0]['target']}")

            elif node_data['type'] == 'OUTPUT':
                # Output 노드는 흐름의 끝이므로 더 이상 진행하지 않음
                print(f"흐름 종료: Output 노드 '{node_to_execute_id}' 도달.")
                break # 흐름 종료

            else:
                # 일반 노드는 연결된 모든 다음 엣지를 따라감 (단순 시뮬레이션)
                # 실제 LangGraph는 다음 상태를 정확히 정의해야 함
                for edge in next_edges:
                    # 엣지에 조건이 없는 경우에만 추가 (라우터가 아닌 일반 노드)
                    if 'condition' not in edge or not edge['condition']:
                        queue.append(edge['target'])
                        print(f"다음 노드: {edge['target']}")
                if not next_edges:
                    print(f"경고: 노드 '{node_to_execute_id}'에 연결된 다음 엣지가 없습니다. 흐름 중단.")
                    break


        print("\n=== 흐름 실행 완료 ===")
        print(f"최종 상태: {current_state}")
        print(f"규칙 위반 엣지 ID: {violated_edges}")
        return current_state, execution_log, violated_edges

# --- JSON 파일 생성 (실행을 위해 필요) ---
# sample_flow.json
sample_flow_content = {
  "nodes": [
    {"id": "node_start", "type": "STATE", "label": "시작 상태", "properties": {"initial_topic": "Web3 AI 에이전트의 미래"}},
    {"id": "node_llm_blog", "type": "LLM", "label": "블로그 글 생성", "properties": {"promptTemplate": "주제: {topic}에 대한 500자 블로그 초안 작성", "temperature": 0.7, "inputVars": "topic", "outputVars": "blog_post_draft"}},
    {"id": "node_rule_check", "type": "RULE_CHECKER", "label": "편향성 검사", "properties": {"ruleSetName": "no_bias", "inputVars": "blog_post_draft", "outputVars": "is_valid"}},
    {"id": "node_router_decision", "type": "ROUTER", "label": "편향성 라우터", "properties": {"conditionLogic": "if is_valid == false -> to node_llm_rewrite", "inputVars": "is_valid", "outputVars": "decision_path"}},
    {"id": "node_llm_rewrite", "type": "LLM", "label": "블로그 글 수정", "properties": {"promptTemplate": "다음 블로그 글에서 편향성을 제거하고 다시 작성: {blog_post_draft}", "temperature": 0.5, "inputVars": "blog_post_draft", "outputVars": "blog_post_final"}},
    {"id": "node_output_publish", "type": "OUTPUT", "label": "블로그 발행", "properties": {"outputType": "Webhook", "targetUrl": "https://agentlayer.com/publish", "sendVars": "blog_post_final"}},
    {"id": "node_output_alert", "type": "OUTPUT", "label": "알림 전송", "properties": {"outputType": "Slack", "targetUrl": "https://hooks.slack.com/alert", "sendVars": "blog_post_draft"}}
  ],
  "edges": [
    {"id": "edge_1", "source": "node_start", "target": "node_llm_blog"},
    {"id": "edge_2", "source": "node_llm_blog", "target": "node_rule_check"},
    {"id": "edge_3", "source": "node_rule_check", "target": "node_router_decision"}, # Completed edge_3
    {"id": "edge_4", "source": "node_router_decision", "target": "node_llm_rewrite", "condition": "decision_path == 'invalid'"}, # Added missing edges and conditions
    {"id": "edge_5", "source": "node_router_decision", "target": "node_output_publish", "condition": "decision_path == 'valid'"},
    {"id": "edge_6", "source": "node_llm_rewrite", "target": "node_output_publish"}
  ]
}


# constitution.json
constitution_content = {
  "rules": [
    {"name": "no_bias", "description": "콘텐츠는 편향성을 포함해서는 안 됩니다.", "logic": "check_for_bias(content)"},
    {"name": "content_length", "description": "블로그 글 길이는 최소 500 단어 이상이어야 합니다.", "logic": "len(content.split()) >= 500"}
  ]
}

# 파일로 저장 (실제 실행 시 필요)
with open('sample_flow.json', 'w', encoding='utf-8') as f:
    json.dump(sample_flow_content, f, indent=2, ensure_ascii=False)

with open('constitution.json', 'w', encoding='utf-8') as f:
    json.dump(constitution_content, f, indent=2, ensure_ascii=False)

print("JSON 파일 생성 완료: sample_flow.json, constitution.json")

# LangGraphRunner 인스턴스 생성 및 실행
runner = LangGraphRunner()
final_state, execution_log, violated_edges = runner.run_flow()

# 결과 출력
print("\n--- 최종 결과 ---")
print(f"최종 상태: {final_state}")
print("\n--- 실행 로그 ---")
for entry in execution_log:
    print(f"노드 ID: {entry['node_id']}, 타입: {entry['node_type']}, 레이블: {entry['node_label']}")
    print(f"  입력 상태: {entry['input_state_before_execution']}")
    print(f"  출력 데이터: {entry['output_data']}")
    print(f"  출력 상태: {entry['output_state_after_execution']}")
print(f"\n규칙 위반 엣지 ID: {violated_edges}")

JSON 파일 생성 완료: sample_flow.json, constitution.json
흐름 정의 로드 완료: 7개 노드, 6개 엣지
헌법 정의 로드 완료: 2개 규칙

--- 노드 실행: [STATE] 시작 상태 (ID: node_start) ---
초기 상태 설정: {'topic': 'Web3 AI 에이전트의 미래'}
다음 노드: node_llm_blog

--- 노드 실행: [LLM] 블로그 글 생성 (ID: node_llm_blog) ---
LLM 출력: 생성된 콘텐츠: '주제: Web3 AI 에이전트의 미래에 대한 500자 블로그 초안 작성' (온도: 0.7)
다음 노드: node_rule_check

--- 노드 실행: [RULE_CHECKER] 편향성 검사 (ID: node_rule_check) ---
규칙 'no_bias' 검사: 위반 (시뮬레이션)
규칙 검사 결과 (is_valid): False
규칙 위반 엣지 기록: edge_3
다음 노드: node_router_decision

--- 노드 실행: [ROUTER] 편향성 라우터 (ID: node_router_decision) ---
라우터 결정: invalid
라우터 결정에 따라 다음 노드: node_llm_rewrite

--- 노드 실행: [LLM] 블로그 글 수정 (ID: node_llm_rewrite) ---
LLM 출력: 생성된 콘텐츠: '다음 블로그 글에서 편향성을 제거하고 다시 작성: 생성된 콘텐츠: '주제: Web3 AI 에이전트의 미래에 대한 500자 블로그 초안 작성' (온도: 0.7)' (온도: 0.5)
다음 노드: node_output_publish

--- 노드 실행: [OUTPUT] 블로그 발행 (ID: node_output_publish) ---
최종 출력 노드: 타입=Webhook, 대상=https://agentlayer.com/publish, 데이터={'blog_post_final': "생성된 콘텐츠: '다음 블로그 글에서 편향성을 제거하고 다시 작성: 생성